# Filtering and Scaling Depth Data

Congratulations on making it to this part of the project. Before we continue, lets review the overarching goals of Depth Mapping and how we plan on getting there.

First, we want to write a function that will take raw depth and RGB values from the Kinect and filter out the object we are trying to map.  For example, if we want to map a water bottle that is between 2 to 4 feet away from the Kinect, we want to filter out all depth values that are not within 2 to 4 feet.


After we filter out data not in our bounds, we want to scale our data from its original size (640 x 480) into the size of our cube (6 x 6).

We already learned how to write Kinect still frames into .csv files, and in this notebook we will first read the data into an array.

Import the math and numpy modules first:

In [1]:
import math
import numpy as np

Next, initialize an empty python list and read from depth_data.csv.  Notice how, in the for loop, we create a list for each line using line.split(',').  Each line in f is originally a string of numbers, for example: '878,877,657'.  line.split(',') creates a python list in the form \['878', '877', '657'].  We then need to convert the string values into integers, which is where list comprehension becomes useful.  Given an example python list named 'list', \[int(i) for i in list] will convert all of the values in 'list' to integers if possible.  (Lookup documentation for [File Input/Output](https://docs.python.org/3/tutorial/inputoutput.html) and [list comprehension](https://www.pythonforbeginners.com/basics/list-comprehensions-in-python) for more help)

In the cell below, given an empty list depth_data, read the data that you wrote to a .csv file in the previous challenge into depth_data.

### Reading Depth Data

In [2]:
depth_data = []
#loading csv file using numpy
d = np.loadtxt('depthcapture.csv',delimiter=',')
#print(d)
#converting depth to integer type and saving to depth_data array
depth_data = d.astype(int)
print(depth_data)

[[2047 2047 2047 ... 2047 2047 2047]
 [2047 2047 2047 ... 2047 2047 2047]
 [2047 2047 2047 ... 2047 2047 2047]
 ...
 [2047 2047 2047 ... 2047 2047 2047]
 [2047 2047 2047 ... 2047 2047 2047]
 [2047 2047 2047 ... 2047 2047 2047]]


Check to make sure that the length of depth_data and the length of the first element of depth_data are 480 and 640, respectively.  These are the dimensions of the Kinect's camera.

Example: 

len(depth_data) == 480

len(depth_data[0]) == 640

In [3]:
len(depth_data) == 480

True

In [4]:
len(depth_data[0]) == 640

True

Create a variable 'raw_depth_data' to save the original values of depth_data before we filter the array.

In [5]:
raw_depth_data = np.copy(depth_data)
print(raw_depth_data)

[[2047 2047 2047 ... 2047 2047 2047]
 [2047 2047 2047 ... 2047 2047 2047]
 [2047 2047 2047 ... 2047 2047 2047]
 ...
 [2047 2047 2047 ... 2047 2047 2047]
 [2047 2047 2047 ... 2047 2047 2047]
 [2047 2047 2047 ... 2047 2047 2047]]


### Filtering Algorithm

Now that we have read our csv file into an array, we want to set bounds for the data we wish to see.  For example, if we want to visualize a water bottle that is between 3 and 5 feet from the Kinect, then we will only want to see data values between 3-5 feet to eliminate any background noise in the data.  Later on we will be visualizing our data and we will see the difference between the filtered and unfiltered data sets. 

At this point we will want depth_data to be a numpy array.  This will allow us to utilize an assortment of numpy functions that will make our algorithm more efficient.  If you are unfamiliar with numpy, refer to its documentation [here](https://docs.scipy.org/doc/numpy-1.16.1/reference/index.html) or check out our numpy tutorial.

In [6]:
#making data into a numpy array
depth_data = np.asarray(depth_data)
#print(depth_data)
depth_data

array([[2047, 2047, 2047, ..., 2047, 2047, 2047],
       [2047, 2047, 2047, ..., 2047, 2047, 2047],
       [2047, 2047, 2047, ..., 2047, 2047, 2047],
       ...,
       [2047, 2047, 2047, ..., 2047, 2047, 2047],
       [2047, 2047, 2047, ..., 2047, 2047, 2047],
       [2047, 2047, 2047, ..., 2047, 2047, 2047]])

We set our upper and lower depth bounds to values in feet that will capture the object we are trying to map.

In [7]:
#both values are in feet
ft_lo_depth = 1.5   # You can set these variables to different values
ft_u_depth = 3

Using the distance formula given earlier, convert the values from feet to 11-bit depth values and assign the result to variables lo_depth and u_depth.

In [8]:
#changed distance formula not sure if correct
#distance(meters) = 0.1236 * tan(rawDisparity / 2842.5 + 1.1863)
#meters to feet: 1m = 3.28084ft
#function obtained from website provided in p2 documentation
def conversion(x):
    x *= 0.3048000097536
    raw = 2842.5*(math.atan(x/0.1236) - 1.1863)
    return raw
#converting value to integer (do not want decimal) and printing to check value
lo_depth = conversion(ft_lo_depth)
print('lo_depth =' %d, lo_depth)
u_depth = conversion(ft_u_depth)
print('u_depth =' %d, u_depth)
math.atan(1/0.1236)

lo_depth = 342.42603838735363
u_depth = 711.023111474549


1.447820030473509

In order to make our algorithm as efficient as possible, we want to avoid for loops that run through every value of our data.  Flatten depth_data using np.flatten

In [9]:
#making depth_data into a 1D array
depth_data = depth_data.flatten()

Now, because our array is one-dimensional, we want to be able to find the coordinates where our depth values are between lo_depth and u_depth.  By coordinates, we mean the x and y position where the depth value would be if it were a 2-dimensional array.  For example, if a value were at the position (2,3) in (rows, columns) of a 2D array with 5 rows and 4 columns, try to figure out what its position would be in a 1D array.  As in, if the 2D array were flattened, what single number would represent index (2,3) in our flattened array.  

Using this example, extrapolate to the shape of raw_depth_data, the original 2D array, and figure out how to map the indices of raw_depth_data in a one dimensional array called 'coordinates'.

Now there are a variety of ways to filter a one dimensional array, but in any case, our filtering array of depth values will need to satisfy the condition of being between our set 'lo_depth' and 'u_depth' bounds.

First, find the values of the indices where the depth_data fits given upper and lower bounds, and wrap the resulting 1D list of values in a numpy array.

In [10]:
#fucntion prints index of valied values, array needs to be flatten
coordinates = np.where((depth_data>lo_depth) & (depth_data<u_depth))
#print(coordinates)

In [11]:
#function prints x,y cordinate pairs of valid values
#don't have to flatten array this way can use raw_depth_data
cordinates2 = list(zip(*np.where((raw_depth_data>lo_depth) & (raw_depth_data<u_depth))))
#print(cordinates2)

In [12]:
#testing if both functions are correct
#index value - (largest mutiple of 640) = col #
#cordinate = (index/640,col#)
#44310-(640*69)


Next, figure out how to filter 'depth_data' such that it is between 'lo_depth' and 'u_depth'

In [13]:
#will keep original values satisfying condition and zero out the rest
depth_data = np.where((depth_data>lo_depth) & (depth_data<u_depth),depth_data,0)
#deleting all occurences of 0 in filtered array
depth_data = depth_data[depth_data != 0]
print(depth_data)
#note filtered array is flattened

[711 711 711 ... 642 641 641]


Once we have filtered out depth values that are not within our set upper and lower bounds, compare the length of the raw_depth_data with depth_data to make sure that depth_data is in fact smaller.  When finding the length of raw_depth_data, consider that it is a 2D array with all rows being the same length and figure out how to calculate the total number of values in a 2D array.

In [14]:
# Compare the lengths of depth_data with raw_depth_data.
print('length of filtered array =' %d, len(depth_data))


length of filtered array = 20766


In [15]:
#raw data is 480,640 array: therefore sieze of full array is 480*640
size_raw_data = len(raw_depth_data)*len(raw_depth_data[0])
print('size of raw data array =' %d, size_raw_data)

size of raw data array = 307200


Continue if your length of depth_data is considerably smaller than the number of values in raw_depth_data.

### Scaling Algorithm

Now that we have created a 1D array that contains our filtered depth values, we want to scale our data in order to display it on a cube.

Let look at one method to scale any given object.  Given the image below, how would we map the water bottle?

<img src="example_image.jpg">

Picture a 3D cube surrounding the water bottle.  In our scaling algorithm, this cube will have the width and height of largest axis of our object (in this case the y-axis/height), and the depth will be as given by our bounds.  Our original bounds for depth set above may not in fact capture this water bottle, but imagine that they do.  What we want to do is take our cube surrounding the water bottle and scale the cube into smaller numbers.

Essentially we are fixing new dimensions for our cube that ultimately enables us to map the image on a physical cube.

Now that we have some intuition for what we wish to accomplish with our scaling algorithm, lets figure out how to scale our cube into smaller dimensions. Lets review what our filtered data looks like.

In [16]:
depth_data

array([711, 711, 711, ..., 642, 641, 641])

Each value in our 1D array is a depth value representing the distance from the Kinect to whatever object you place in front of the Kinect.

First, lets find the x and y coordinates for each depth (z) value so that we can eventually transform our x, y, and z to the dimensions of the cube.

In [17]:
#separating x and y from coordinates2
x,y = zip(*cordinates2)
x_arr = np.asarray(x)                  # All the x-values (from 0 to 639) where depth_data fits the depth bounds
y_arr = np.asarray(y)                  # All the y-values (from 0 to 479) where depth_data fits the depth bounds
z_arr = depth_data    # All the z or depth values (11-bit) where depth_data fits the depth bounds
print(x_arr)
max(x_arr)

[118 118 118 ... 367 367 367]


367

We will store the depth data under a new variable to be used in visualizing the data later.  This will be done by creating a list of tuples using [set()](https://docs.python.org/2/library/sets.html) and [zip()](https://docs.python.org/3/library/functions.html#zip) functions.

In [18]:
raw_tuple_depth = list(zip(x_arr,y_arr,z_arr))
raw_tuple_depth = np.asarray(raw_tuple_depth)
print(raw_tuple_depth)


[[118  35 711]
 [118  36 711]
 [118  37 711]
 ...
 [367  70 642]
 [367  71 641]
 [367  72 641]]


Find bounds for each dimension (x, y, z) and store them under new variables.

In [20]:
r  = max(x_arr)   # Rightmost x-value (max)
le = min(x_arr)   # Leftmost x-value (min)
u  = min(y_arr)   # Upper or smallest y-value (min)
lo = max(y_arr)   # Lower or largest y-value (max)
b  = max(z_arr)   # Maximum z-value
f  = min(z_arr)   # Minimum z-value
print(r, le, u, lo, b, f)

367 118 0 121 711 598


We can now use the bounds of our object to scale our data.  Here is what this would look like in our example image:

<img src="example_image_pt2.jpg">

raw_tuple_depth = ...
raw_tuple_depth

In [21]:
x_range = r - le  # potential legnth of cube
y_range = lo - u  # potential length of cube
print('x =' %d, x_range)
print('y =' %d, y_range)

x = 249
y = 121


Looking at the image above, it intuitively makes sense to map our object by forming a square around the largest range between x and y.  In this case the y-range is largest.

In [22]:
max_range = max(x_range,y_range)   # find the maximum range between x and y.  A square will be formed around the maximum range which our object will be placed in
x_mid = r - x_range/2    # calculate the midpoint of the x-axis of the object
y_mid = lo - y_range/2    # calculate the midpoint of the y-axis of the object
print(max_range,x_mid, y_mid)

249 242.5 60.5


In [23]:
y_mid


60.5

Create a conditional below that will change upper bounds (u) or left bounds (le) based on if the x_range is larger or if the y_range is larger, respectively.  You can see what this will accomplish in the image below.

In [24]:
#when running sometimes get negative value for u. Not sure if correct or should be zero
# Make an if else statement
if x_range > y_range:
    u = y_mid-max_range/2
    lo = y_mid+max_range/2
elif x_range < y_range:
    le = x_mid-max_range/2
    r = x_mid+max_range/2
print(u, lo)
print(lo-u)

-64.0 185.0
249.0


Here is what our x-y mapping should look like:

<img src="example_image_pt3.jpg">

As for the z_range of our cube, we will use the distance between 'u_depth' and 'lo_depth'.

In [25]:
z_range = u_depth - lo_depth

In [26]:
z_range

368.5970730871953

Create a variable 'cube_length' that will represent the length of our cube for this project.

In [27]:
cube_length = 6
x_arr = np.asarray(x_arr)
y_arr = np.asarray(y_arr)
z_arr = np.asarray(z_arr)

Now we have all the components we need to scale 'depth_data' into our cube's values.  The resulting x, y, and z arrays should have values between 0 and the cube_length.

Try drawing a picture for this section to figure out how to scale our 3D cube from one set of dimensions into another.  Consider the picture above.

Variables 'x_new', 'y_new', and 'z_new' will represent our x, y, and z values in the dimensions of our cube. Ideally, we want to create these arrays using matrix manipulation allowed by numpy.  This means creating arrays and applying arithmetic operations on them to avoid using for loops.  This will make our algorithm much more efficient.

Hint: You will need to create additional variables below to scale the arrays and avoid for loops.  [This](https://www.pluralsight.com/guides/overview-basic-numpy-operations) article details how to apply arithmetic operations between numpy arrays.

In [28]:
#need to normalize individual x, y, and z coordinates from 0 to 5.
x_scaled = ((x_arr - le)*(cube_length-1))/max_range
x_scaled

array([0., 0., 0., ..., 5., 5., 5.])

In [29]:
y_scaled = ((y_arr - u)*(cube_length-1))/max_range
y_scaled.max()

3.714859437751004

In [30]:
z_scaled = ((z_arr - lo_depth)*(cube_length-1))/(z_range)
z_scaled

array([4.99968649, 4.99968649, 4.99968649, ..., 4.06370511, 4.05014016,
       4.05014016])

In [31]:
# Create arrays x_new, y_new, and z_new that represent the x, y, and z values of our cube
x_new = x_scaled
y_new = y_scaled
z_new = z_scaled

We will now be able to create our array of values that will map our object by combining x_new, y_new, and z_new arrays.  The output will be a list of tuples.

In [32]:
filtered_depth = list(zip(x_new,y_new,z_new))
filtered_depth

[(0.0, 1.9879518072289157, 4.999686494057652),
 (0.0, 2.0080321285140563, 4.999686494057652),
 (0.0, 2.0281124497991967, 4.999686494057652),
 (0.020080321285140562, 1.9879518072289157, 4.999686494057652),
 (0.020080321285140562, 2.0080321285140563, 4.986121546408659),
 (0.020080321285140562, 2.0281124497991967, 4.972556598759666),
 (0.020080321285140562, 2.0481927710843375, 4.958991651110672),
 (0.020080321285140562, 2.068273092369478, 4.945426703461679),
 (0.020080321285140562, 2.0883534136546187, 4.931861755812685),
 (0.020080321285140562, 2.108433734939759, 4.904731860514699),
 (0.040160642570281124, 1.9678714859437751, 4.999686494057652),
 (0.040160642570281124, 1.9879518072289157, 4.999686494057652),
 (0.040160642570281124, 2.0080321285140563, 4.986121546408659),
 (0.040160642570281124, 2.0281124497991967, 4.972556598759666),
 (0.040160642570281124, 2.0481927710843375, 4.958991651110672),
 (0.040160642570281124, 2.068273092369478, 4.945426703461679),
 (0.040160642570281124, 2.0883

As a final check, make sure that the sets of your x, y, and z arrays are within the values 0 to the length of the cube.

In [43]:
filtered_depth = filtered_depth.astype(int)
filtered_depth

array([[0, 1, 4],
       [0, 2, 4],
       [0, 2, 4],
       ...,
       [5, 2, 4],
       [5, 2, 4],
       [5, 2, 4]])

# Visualizing Data

In this section we will be using sklearn.cluster and plotly to visualize our data.

In [34]:
# Make sure you have these following packages installed
#from sklearn.cluster import SpectralClustering, KMeans
import plotly.offline as py
from plotly import graph_objs as go
#to install plotly run 
    #sudo pip install plotly

Lets look at our raw_tuple_depth variable we created earlier.  We will first visualize our raw data and compare that to our mapped data.

In [35]:
#raw_tuple_depth
raw_tuple_depth = np.array(raw_tuple_depth)
filtered_depth = np.array(filtered_depth)

Put the name of your fi

In [41]:
#plots = []
#x, y , z = zip(*raw_tuple_depth)
x, y, z = raw_tuple_depth[:, 0], raw_tuple_depth[:, 1], raw_tuple_depth[:, 2]
plots = [go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2), connectgaps=False)] #,color=c[k]

file_name = 'p2_hardware.ipynb'  #the name of your file
py.plot(plots,filename=file_name,auto_open=True)

'p2_hardware.ipynb.html'

Now lets look at our updated 'filtered_depth'.

In [46]:
#x, y , z = zip(*filtered_depth)
x, y, z = filtered_depth[:, 0], filtered_depth[:, 1], filtered_depth[:, 2]
plots = [go.Scatter3d(x=x, y=y, z=z, mode='markers', marker=dict(size=2), connectgaps=False)]
file_name = 'p2_hardware.html' #the name of your file
py.plot(plots,filename=file_name,auto_open=True)

'p2_hardware.html'

In [45]:
#ploting heat map
x, y, z = filtered_depth[:, 0], filtered_depth[:, 1], filtered_depth[:, 2]
plots = [go.Heatmap(x=x, y=y, z=z)]
file_name = 'p2_hardware_heatmap.html' #the name of your file
py.plot(plots,filename=file_name,auto_open=True)

'p2_hardware_heatmap.html'

** If you want more information as to how these plots were generated, refer to [plotly](https://plot.ly/python/user-guide/) documentation